## Notebook demonstrating the importing of terrain layers from SRTM or 3DEP datasets for any given region in the contiguous US using ssrs package

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from ssrs import Terrain, TurbinesUSWTB
from ssrs.raster import transform_bounds, transform_coordinates, get_raster_in_projected_crs
from ssrs.utils import get_extent_from_bounds, create_gis_axis

ModuleNotFoundError: No module named 'richdem'

In [ ]:
# directory where output is saved
output_dir = os.path.join(os.path.abspath(os.path.curdir), 'output/')

In [ ]:
# Parameters for determining the region of interest
proj_crs = 'ESRI:102008' # projected crs
lonlat_crs = 'EPSG:4326' # geo crs
southwest_lonlat = (-106.14, 42.77) # southwestern point
region_width_km = (60., 50.) # region size in km
resolution = 100. # resolution in meters

In [ ]:
# figure out the grid size of the target region
xsize = int(round((region_width_km[0] * 1000. / resolution)))
ysize = int(round((region_width_km[1] * 1000. / resolution)))
gridsize = (ysize, xsize)

In [ ]:
# get the bounds of the revion in both lonlat and projected crs
proj_west, proj_south = transform_coordinates(lonlat_crs, proj_crs, southwest_lonlat[0], southwest_lonlat[1])
proj_east = proj_west[0] + xsize * resolution
proj_north = proj_south[0] + ysize * resolution
bounds = (proj_west[0], proj_south[0], proj_east, proj_north)
extent = get_extent_from_bounds(bounds)
lonlat_bounds = transform_bounds(bounds, proj_crs, lonlat_crs)

In [ ]:
#Valid terrain layers
Terrain.valid_layers

In [ ]:
# downloading terrain layers
terrain_layers = {
    'Elevation': 'DEM',
    'Slope': 'Slope Degrees',
    'Aspect': 'Aspect Degrees'
}
region = Terrain(lonlat_bounds, output_dir)
region.download(terrain_layers.values())

In [ ]:
def get_terrain_layer(lyr:str):
    fpath = region.get_raster_fpath(lyr)
    return get_raster_in_projected_crs(fpath, bounds, gridsize, resolution, proj_crs)

In [ ]:
# get all the wind turbines in this region
turbines = TurbinesUSWTB(bounds, proj_crs, min_hubheight=60.)
turb_xlocs, turb_ylocs = turbines.get_locations()
turbines.print_details()

In [ ]:
# plot all the layers
for key, val in terrain_layers.items():
    lyr_data = get_terrain_layer(val)
    fig, ax = plt.subplots(figsize=(6,5))
    cm = ax.imshow(lyr_data, cmap='terrain', extent=extent, origin='lower')
    create_gis_axis(fig, ax, cm)
    ax.plot(turb_xlocs, turb_ylocs, '1k', alpha=0.75, markersize=3.)
    ax.set_title(key)